This submission will eventually become your Introduction/Business Problem section in your final report. So I recommend that you push the report (having your Introduction/Business Problem section only for now) to your Github repository and submit a link to it.

* Where is the best place to open a restaurant in London Ontario? Where are the most reviewed and what seems to be the most popular. And also ehat type of restaurant is more popular? This problem will be solved by using Foursquare API (view user count and tip count, etc.). This question will help everyone who wants to open a restaurant in London Ontario and wants to know if this is feasible.

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

In [1]:
!pip install geocoder

     |████████████████████████████████| 98 kB 7.1 MB/s  eta 0:00:01


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                    -failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.9|3.6.9|3.6.9|3.6.9|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|>=2.7,<2.8.0a0|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|1_73_pypy|2_73_pypy|3_73_pypy']
  - rsa -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']

Your python: python=3.7

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, th

ModuleNotFoundError: No module named 'folium'

In [4]:
address = 'London , Canada'

geolocator = Nominatim(user_agent="LNON_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London Ontario is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London Ontario is 42.9836747, -81.2496068.


In [5]:

# Set up Foursquare
CLIENT_ID = 'G0XWNCZRXQ1FQW31X12IGCE4PXZOA3HURG2L1WSQGJFXPYHU' # your Foursquare ID
CLIENT_SECRET = 'W42QEEFCFIOYZUETK0ME3VPJMUE44UMWYBFDZBICVRVVQP1F' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G0XWNCZRXQ1FQW31X12IGCE4PXZOA3HURG2L1WSQGJFXPYHU
CLIENT_SECRET:W42QEEFCFIOYZUETK0ME3VPJMUE44UMWYBFDZBICVRVVQP1F


In [6]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=G0XWNCZRXQ1FQW31X12IGCE4PXZOA3HURG2L1WSQGJFXPYHU&client_secret=W42QEEFCFIOYZUETK0ME3VPJMUE44UMWYBFDZBICVRVVQP1F&ll=42.9836747,-81.2496068&v=20180605&query=Restaurant&radius=50000&limit=100'

In [8]:

results = requests.get(url).json()

In [9]:

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4b97329ff964a520cbfb34e3,Thaifoon Restaurant,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",v-1607477323,False,120 Dundas St.,near Talbot St.,42.983311,-81.251357,"[{'label': 'display', 'lat': 42.98331116257564...",148,N6A 1G3,CA,London,ON,Canada,"[120 Dundas St. (near Talbot St.), London ON N...",62339564,NaN
1,4b61ce40f964a520fa232ae3,Tahinis Restaurant,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",v-1607477323,False,11-551 Richmond St,at Albert St.,42.988600,-81.250732,"[{'label': 'display', 'lat': 42.9885998, 'lng'...",555,N6A 3E9,CA,London,ON,Canada,"[11-551 Richmond St (at Albert St.), London ON...",561310659,NaN
2,4e74e1fb8998ed82a43fcaf0,Melody Restaurant,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1607477323,False,NaN,NaN,42.983746,-81.249484,"[{'label': 'display', 'lat': 42.983746, 'lng':...",12,NaN,CA,London,ON,Canada,"[London ON, Canada]",NaN,NaN
3,4b5e63e5f964a520ba8c29e3,Symposium Cafe Restaurant & Lounge,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1607477323,False,620 Richmond St.,at Central Ave.,42.990655,-81.250828,"[{'label': 'display', 'lat': 42.990655149613, ...",783,N6J 5A9,CA,London,ON,Canada,"[620 Richmond St. (at Central Ave.), London ON...",146445443,NaN
4,4bf532dd706e20a1e222aa98,Westside Family Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1607477323,False,107 Mount Pleasant Ave,Wharncliffe Rd. N.,42.984838,-81.263275,"[{'label': 'display', 'lat': 42.98483774629121...",1120,N6H 1E1,CA,London,ON,Canada,"[107 Mount Pleasant Ave (Wharncliffe Rd. N.), ...",NaN,NaN


In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Thaifoon Restaurant,Thai Restaurant,120 Dundas St.,near Talbot St.,42.983311,-81.251357,"[{'label': 'display', 'lat': 42.98331116257564...",148,N6A 1G3,CA,London,ON,Canada,"[120 Dundas St. (near Talbot St.), London ON N...",NaN,4b97329ff964a520cbfb34e3
1,Tahinis Restaurant,Middle Eastern Restaurant,11-551 Richmond St,at Albert St.,42.988600,-81.250732,"[{'label': 'display', 'lat': 42.9885998, 'lng'...",555,N6A 3E9,CA,London,ON,Canada,"[11-551 Richmond St (at Albert St.), London ON...",NaN,4b61ce40f964a520fa232ae3
2,Melody Restaurant,Diner,NaN,NaN,42.983746,-81.249484,"[{'label': 'display', 'lat': 42.983746, 'lng':...",12,NaN,CA,London,ON,Canada,"[London ON, Canada]",NaN,4e74e1fb8998ed82a43fcaf0
3,Symposium Cafe Restaurant & Lounge,Restaurant,620 Richmond St.,at Central Ave.,42.990655,-81.250828,"[{'label': 'display', 'lat': 42.990655149613, ...",783,N6J 5A9,CA,London,ON,Canada,"[620 Richmond St. (at Central Ave.), London ON...",NaN,4b5e63e5f964a520ba8c29e3
4,Westside Family Restaurant,Breakfast Spot,107 Mount Pleasant Ave,Wharncliffe Rd. N.,42.984838,-81.263275,"[{'label': 'display', 'lat': 42.98483774629121...",1120,N6H 1E1,CA,London,ON,Canada,"[107 Mount Pleasant Ave (Wharncliffe Rd. N.), ...",NaN,4bf532dd706e20a1e222aa98


In [12]:
!pip install folium

     |████████████████████████████████| 93 kB 3.0 MB/s  eta 0:00:01


In [13]:
import folium

In [15]:

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 


folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='London Ontario',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map